In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('data/Titanic.csv', sep=',')
df.head()

,Name,PClass,Age,Sex,Survived
0,"Allen, Miss Elisabeth Walton",1st,29.00,female,1
1,"Allison, Miss Helen Loraine",1st,2.00,female,0
2,"Allison, Mr Hudson Joshua Creighton",1st,30.00,male,0
3,"Allison, Mrs Hudson JC (Bessie Waldo Daniels)",1st,25.00,female,0
4,"Allison, Master Hudson Trevor",1st,0.92,male,1


In [ ]:
# Количество точек данных (измерений) в наборе

In [ ]:
# Количество полей данных (атрибутов)

In [ ]:
# Тип данных каждого поля, шкала каждого поля

In [ ]:
# Количество отсутствующих значений для каждого поля

In [ ]:
# Количество и названия константных и уникальных полей